In [1]:
from mobo.surrogate_model import BoTorchSurrogateModelReapeat, BoTorchSurrogateModel
from mobo.transformation import StandardTransform
import numpy as np

In [6]:
import torch
import pandas as pd
from pathlib import Path
sm = BoTorchSurrogateModelReapeat(n_var=1, n_obj=1, alpha=0.9, n_w=11)

# .\result\exp4d\optimization_4\marsdet\0\state_dict_noise.pt

path = Path("./result/exp4d/optimization_4/marsdet/0/")

state_dict = torch.load( path / "state_dict.pt")
state_dict_noise = torch.load( path / "state_dict_noise.pt")

eval_samples = pd.read_csv(path / "EvaluatedSamples.csv")
eval_samples

,iterID,x1,x2,x3,x4,f1,rho_f1,mvar_f1,f2,rho_f2,...,Uncertainty_f1,Acquisition_f1,Expected_f2,Uncertainty_f2,Acquisition_f2,Expected_f3,Uncertainty_f3,Acquisition_f3,Hypervolume_indicator,MVaR_Hypervolume_indicator
0,0,1.760000,0.280000,4.030000,2.32000,-0.230135,0.010827,-0.096784,-3.473333,0.619033,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
1,0,2.590000,0.660000,9.150000,1.09000,-0.909779,0.003111,-0.838301,-2.293333,0.002533,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
2,0,0.800000,0.860000,6.920000,1.93000,-0.045496,0.000147,-0.029955,-0.760000,0.127300,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
3,0,2.750000,0.510000,7.890000,1.66000,-0.932535,0.000047,-0.923764,-2.373333,0.035233,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
4,0,1.220000,0.350000,9.480000,1.47000,-0.030110,0.000007,-0.026708,-1.163333,0.028633,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
5,0,3.170000,0.930000,5.140000,1.95000,-0.938840,0.000341,-0.915175,-2.280000,0.001600,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
6,0,1.440000,0.700000,7.560000,1.29000,-0.043314,0.000027,-0.036645,-1.183333,0.061633,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
7,0,2.270000,0.100000,5.810000,2.31000,-0.137306,0.014902,0.019138,-3.930000,2.520700,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
8,0,0.650000,0.180000,7.060000,2.05000,-0.082687,0.009756,0.043895,-4.190000,19.202700,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
9,0,2.990000,0.770000,6.070000,1.54000,-0.936913,0.000137,-0.921935,-2.360000,0.010800,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,22.453525,19.799388
